In [222]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from numpy import linalg as LA
from abc import abstractmethod

# # suppress warnings
# import warnings
# warnings.filterwarnings("ignore")

plt.style.use('dark_background')

In [223]:
class MatrixSolver:
    # compute gradinent of function f in point x with step h
    @staticmethod
    def grad(x, f, eps):
        derivative = np.zeros(np.size(x))
        for i in range(np.size(x)):
            x[i] += eps
            f1 = f(x)
            x[i] -= 2 * eps
            f2 = f(x)
            x[i] += eps
            derivative[i] = (f1 - f2) / (2 * eps)
        return derivative

    # compute Jacobian of functions rs = (r_1...r_n), r_i = r_i(x_1...x_m)
    @staticmethod
    def findJacobian(rs, x, eps):
        n = np.size(rs)
        m = np.size(x)
        J = np.zeros((n, m))
        for i in range(n):
            J[i] = MatrixSolver.grad(x, rs[i], eps)
        return J

    # compute pseudo inverse of matrix X
    @staticmethod
    def pseudoInverse(x):
        return np.linalg.inv(x.T @ x) @ x.T

    # compute hessian of functions R^n -> R
    @staticmethod
    def findHessian(func, x, eps):
        def grad_f(x):
            return (MatrixSolver.grad(x, func, eps))

        return MatrixSolver.findJacobian([grad_f], x, eps)


In [224]:
class Searcher:
    def __init__(self, eps, max_iterations, func, start_x):
        self.eps = eps
        self.max_iterations = max_iterations
        self.func = func
        self.start_x = start_x

    # stop method
    def is_not_final(self):
        return LA.norm(self.next - self.cur_x) > self.eps and self.max_iterations > self.epoch

    # executor method
    def run(self):
        self.epoch = 0
        #self.lr = start_lr
        self.cur_x = self.start_x
        #self.lr_func = lr_func
        #self.y = y
    
        self.next_point()
        while (self.is_not_final()):
            self.epoch = self.epoch + 1
            self.cur_x = self.next
            self.next_point()
        return self.next


    # find next point
    @abstractmethod
    def next_point(self):
        pass

    


In [225]:
class Dogleg(Searcher):

    def __init__(self, eps, max_iteration, func, start_x,
                 initial_trust_radius=1.0, max_trust_radius=100.0, eta = 0.15):
        super().__init__(eps, max_iteration, func, start_x)
        self.initial_trust_radius = initial_trust_radius
        self.max_trust_radius = max_trust_radius
        self.eta = eta

    def find_shift(self):
        # find optimum
        optimum = -(np.linalg.inv(self.hess) @ self.jac)
        norm_opt = sqrt(optimum @ optimum)

        if norm_opt <= self.trust_radius:
            return optimum

        # find Cauchy point
        cauchy = - (self.jac @ self.jac / (self.jac @ (self.hess @ self.jac))) * self.jac
        norm_cauchy = sqrt(cauchy @ cauchy)

        # stop in circul
        if norm_cauchy >= self.trust_radius:
            return self.trust_radius * cauchy / norm_cauchy

        tau = (self.trust_radius - cauchy) / (optimum - cauchy)

        return cauchy + tau * (optimum - cauchy)

    def find_trust_radius(self, shift):
        act_reduction = self.func(self.cur_x) - self.func(self.cur_x + shift)
        pred_reduction = -(self.jac @ shift + 0.5 * (shift @ (self.hess @ shift)))

        rho = act_reduction / pred_reduction
        if pred_reduction == 0.0:
            rho = 1e99

        norm_shift = sqrt(shift @ shift)

        if rho < 0.25:
            self.trust_radius = 0.25 * norm_shift
        else:
            if rho > 0.75 and norm_shift == self.trust_radius:
                self.trust_radius = min(2.0 * self.trust_radius, self.max_trust_radius)
            else:
                self.trust_radius = self.trust_radius

        if rho > self.eta:
            self.next = self.cur_x + shift
        else:
            # too close
            self.next = self.cur_x


    def next_point(self):
        self.jac = MatrixSolver.findJacobian([self.func], self.cur_x, self.eps)
        self.hess = MatrixSolver.findHessian(self.func, self.cur_x, self.eps)
        self.trust_radius = self.initial_trust_radius

        self.find_trust_radius(self.find_shift())





In [226]:
class GaussNewton(Searcher):
  
    def next_point(self):
        self.lr = self.lr_func(self.lr)
        p = MatrixSolver.pseudoInverse(MatrixSolver.findJacobian(self.func, self.cur_x, self.eps))
        self.next_x = self.cur_x - self.lr * np.dot(p, self.y).reshape((-1))



In [227]:
def f(x):
    return 0.4*x[0] + 4.7*sin(x[0]) + 2

print(Dogleg(1e-5, 1000, f, np.array([3.2], float)).run())

[[4.62717952]]


# Новый раздел